In [1]:
import tensorflow as tf
from keras.applications import Xception
from keras.utils import multi_gpu_model
import numpy as np

Using TensorFlow backend.


In [2]:
num_samples = 1000
height = 224
width = 224
num_classes = 1000

In [3]:
# Instantiate the base model (or "template" model).
# We recommend doing this with under a CPU device scope,
# so that the model's weights are hosted on CPU memory.
# Otherwise they may end up hosted on a GPU, which would
# complicate weight sharing.

with tf.device('/cpu:0'):
    model = Xception(weights=None,
                     input_shape=(height, width, 3),
                     classes=num_classes)

In [4]:
# Replicates the model on 8 GPUs.
# This assumes that your machine has 8 available GPUs.
parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [5]:
# Generate dummy data.
x = np.random.random((num_samples, height, width, 3))
y = np.random.random((num_samples, num_classes))

In [6]:
# This `fit` call will be distributed on 8 GPUs.
# Since the batch size is 256, each GPU will process 32 samples.
parallel_model.fit(x, y, epochs=10, batch_size=256)

Epoch 1/10
1000/1000 [==============================] - 72s 72ms/step - loss: 3484.9902
Epoch 2/10
1000/1000 [==============================] - 7s 7ms/step - loss: 3452.7862
Epoch 3/10
1000/1000 [==============================] - 7s 7ms/step - loss: 3452.2452
Epoch 4/10
1000/1000 [==============================] - 7s 7ms/step - loss: 3451.5820
Epoch 5/10
1000/1000 [==============================] - 7s 7ms/step - loss: 3450.9147
Epoch 6/10
1000/1000 [==============================] - 8s 8ms/step - loss: 3450.9049
Epoch 7/10
1000/1000 [==============================] - 8s 8ms/step - loss: 3448.6210
Epoch 8/10
1000/1000 [==============================] - 8s 8ms/step - loss: 3445.5065
Epoch 9/10
1000/1000 [==============================] - 8s 8ms/step - loss: 3442.4528
Epoch 10/10
1000/1000 [==============================] - 7s 7ms/step - loss: 3439.5177


In [7]:
# Save model via the template model (which shares the same weights):
model.save('my_model.h5')